<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/test_notebooks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install --upgrade transformers datasets captum quantus accelerate

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# ===================================================
# KONFIGURACJA TRENINGOWA
# ===================================================

# === Ścieżki danych i modelu ===
DATA_PATH = "/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv"  # Plik CSV z danymi Jigsaw Toxic Comment
OUTPUT_MODEL_DIR = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-full"  # Katalog zapisu wytrenowanego modelu

# === Parametry modelu ===
MODEL_CHECKPOINT = "distilbert-base-uncased"  # Model bazowy do fine-tuningu (uncased = nie rozróżnia wielkich/małych liter)
MAX_SEQUENCE_LENGTH = 256  # Maksymalna długość sekwencji tokenów (256 wystarczy dla komentarzy; max dla DistilBERT to 512)

# === Hiperparametry treningu ===
BATCH_SIZE = 16  # Rozmiar batcha treningowego (zwiększ do 32 dla GPU z większą pamięcią np. A100; dla T4 w Colab Free 16 jest bezpieczne)
NUM_EPOCHS = 3  # Liczba epok treningu (standard dla BERT to 2-4 epoki; więcej = ryzyko overfittingu)
LEARNING_RATE = 2e-5  # Learning rate (2e-5 jest standardem dla fine-tuningu BERT; zmiana może destabilizować trening)
WEIGHT_DECAY = 0.01  # Regularyzacja L2 zapobiegająca overfittingowi


In [ ]:
# ===================================================
# 1. PRZYGOTOWANIE DANYCH
# ===================================================


def clean_text(example):
    """
    Czyści tekst komentarza, usuwając szum i normalizując format.

    Funkcja stosowana zarówno podczas treningu jak i ewaluacji, aby zapewnić
    spójność przetwarzania danych.

    Argumenty:
        example: Słownik zawierający klucz 'comment_text' z tekstem do oczyszczenia

    Zwraca:
        Zmodyfikowany słownik example z oczyszczonym tekstem

    Operacje czyszczenia:
        - Konwersja na małe litery (wymagane dla uncased BERT)
        - Usunięcie URL (http/https/www)
        - Usunięcie adresów IP
        - Normalizacja białych znaków (zamiana \\n na spacje, collapse wielokrotnych spacji)
    """
    text = example["comment_text"]
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", text)
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text).strip()
    example["comment_text"] = text
    return example


print(">>> Wczytywanie danych...")
# Wczytanie pełnego zbioru danych (jeśli trwa za długo, użyj .sample(n=50000) lub .head(10000))
df = pd.read_csv(DATA_PATH)

# Ograniczenie wielkości zbioru dla szybszego treningu (odkomentuj jeśli potrzebne)
df = df.head(10000)  # Trening na 10k próbek (usuń tę linię dla pełnego treningu)

dataset = Dataset.from_pandas(df)
dataset = dataset.map(clean_text)

# === Tokenizacja ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)


def tokenize_function(examples):
    """
    Tokenizuje teksty do formatu akceptowanego przez model BERT.

    Argumenty:
        examples: Batch przykładów z kluczem 'comment_text'

    Zwraca:
        Słownik z kluczami: input_ids, attention_mask

    Parametry tokenizacji:
        - padding="max_length": Wyrównuje wszystkie sekwencje do MAX_SEQUENCE_LENGTH
        - truncation=True: Obcina zbyt długie teksty
        - max_length=256: Długość sekwencji (równowaga kontekst/szybkość)
    """
    return tokenizer(
        examples["comment_text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_SEQUENCE_LENGTH,
    )


print(">>> Tokenizacja...")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# === Przygotowanie etykiet multi-label ===
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]


def format_labels(example):
    """
    Konsoliduje 6 osobnych kolumn binarnych w jedną listę etykiet.

    Argumenty:
        example: Słownik z osobnymi kolumnami etykiet

    Zwraca:
        Zmodyfikowany example z kluczem 'labels' zawierającym listę float
    """
    example["labels"] = [float(example[col]) for col in label_cols]
    return example


tokenized_dataset = tokenized_dataset.map(format_labels)

# Usunięcie zbędnych kolumn (zachowanie tylko danych potrzebnych modelowi)
tokenized_dataset = tokenized_dataset.remove_columns(
    ["id", "comment_text"] + label_cols
)
tokenized_dataset.set_format("torch")

# === Podział na zbiór treningowy i walidacyjny ===
splits = tokenized_dataset.train_test_split(
    test_size=0.1, seed=42
)  # 10% na walidację, 90% na trening
train_ds = splits["train"]
eval_ds = splits["test"]

print(
    f"Dane gotowe. Zbiór treningowy: {len(train_ds)}, Zbiór walidacyjny: {len(eval_ds)}"
)


In [ ]:
# ===================================================
# 2. DEFINICJA MODELU I METRYK
# ===================================================

# === Załadowanie modelu ===
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=6,  # 6 kategorii toksyczności (toxic, severe_toxic, obscene, threat, insult, identity_hate)
    problem_type="multi_label_classification",  # Multi-label (komentarz może mieć wiele etykiet jednocześnie)
)


def compute_metrics(p):
    """
    Oblicza metryki ewaluacyjne dla klasyfikacji multi-label.

    Argumenty:
        p: EvalPrediction object zawierający:
            - predictions: Logity z modelu (przed sigmoidem) shape [batch_size, 6]
            - labels: Prawdziwe etykiety binarne shape [batch_size, 6]

    Zwraca:
        Słownik z metrykami: f1_micro, f1_macro, accuracy, roc_auc

    Wyjaśnienie metryk:
        - f1_micro: Globalny F1 (traktuje wszystkie etykiety jednakowo; dobry dla niezbalansowanych danych)
        - f1_macro: Średnia F1 per klasa (równa waga dla każdej kategorii)
        - accuracy: Dokładność per przykład (wszystkie etykiety muszą się zgadzać)
        - roc_auc: Area Under ROC Curve (mierzy jakość rankingu prawdopodobieństw)

    Uwagi:
        - Używamy sigmoid do konwersji logits -> prawdopodobieństwa (BCEWithLogitsLoss już zawiera sigmoid)
        - Próg klasyfikacji = 0.5 (probability > 0.5 => etykieta pozytywna)
    """
    predictions, labels = p

    # Sigmoid do konwersji logits na prawdopodobieństwa (zakres 0-1)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.tensor(predictions))
    preds = (probs > 0.5).int().numpy()  # Binaryzacja przez próg 0.5

    # Obliczanie metryk
    f1_micro = f1_score(labels, preds, average="micro")
    f1_macro = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)

    try:
        roc_auc = roc_auc_score(labels, probs, average="macro")
    except:
        roc_auc = 0.0  # Zabezpieczenie gdy w batchu brakuje przykładów z daną klasą

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "accuracy": acc,
        "roc_auc": roc_auc,
    }


In [ ]:
# ===================================================
# 3. KONFIGURACJA I URUCHOMIENIE TRENINGU
# ===================================================

training_args = TrainingArguments(
    output_dir=f"{OUTPUT_MODEL_DIR}_checkpoints",  # Katalog dla checkpointów (pośrednich zapisów modelu)
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=WEIGHT_DECAY,  # Regularyzacja L2 zapobiegająca overfittingowi
    eval_strategy="epoch",  # Ewaluacja po każdej epoce
    save_strategy="epoch",  # Zapis checkpointu po każdej epoce
    logging_steps=5,  # Logowanie co 5 kroków treningowych
    load_best_model_at_end=True,  # Po treningu załaduj najlepszy model (według metryki metric_for_best_model)
    metric_for_best_model="f1_micro",  # Kryterium wyboru najlepszego modelu (f1_micro dobrze działa dla multi-label)
    save_total_limit=2,  # Zachowaj tylko 2 ostatnie checkpointy (oszczędność miejsca na dysku)
    fp16=True,  # Mixed precision training (przyspieszenie na GPU; wymaga CUDA)
    report_to="none",  # Wyłączenie integracji z Weights & Biases
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print(">>> Rozpoczynanie treningu...")
print(
    f"Parametry: {NUM_EPOCHS} epok, batch size {BATCH_SIZE}, learning rate {LEARNING_RATE}"
)
trainer.train()


In [ ]:
# ===================================================
# 4. ZAPIS FINALNEGO MODELU I WYNIKÓW
# ===================================================

print(f">>> Zapisywanie modelu do: {OUTPUT_MODEL_DIR}")
trainer.save_model(OUTPUT_MODEL_DIR)  # Zapis modelu (wagi + konfiguracja)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)  # Zapis tokenizera (słownik + konfiguracja)

# === Ewaluacja finalna na zbiorze walidacyjnym ===
metrics = trainer.evaluate()
print("Metryki finalne:", metrics)

# Zapis metryk do pliku tekstowego (dla dokumentacji)
with open(f"{OUTPUT_MODEL_DIR}/training_results.txt", "w", encoding="utf-8") as f:
    f.write("=== WYNIKI TRENINGU ===\n")
    f.write(str(metrics))

print("\n>>> Trening zakończony pomyślnie!")
print(f"Model zapisany w: {OUTPUT_MODEL_DIR}")
